In [1]:
%matplotlib ipympl
from matplotlib import pylab as plt
import matplotlib
import molpx
import mdtraj as md

import numpy as np
import pandas as pd

import pytraj as pt
import pyemma

#import matplotlib 
#import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import glob

_ColormakerRegistry()

## Load reference structures

In [2]:
rf0=pt.load('./parm_rst/closed6_dry_rot0.rst7', './parm_rst/closed6_dry_rot0.parm7')
rf1=pt.load('./parm_rst/closed6_dry_rot1.rst7', './parm_rst/closed6_dry_rot1.parm7')
rf2=pt.load('./parm_rst/closed6_dry_rot2.rst7', './parm_rst/closed6_dry_rot2.parm7')
rf3=pt.load('./parm_rst/closed6_dry_rot3.rst7', './parm_rst/closed6_dry_rot3.parm7')
rf4=pt.load('./parm_rst/closed6_dry_rot4.rst7', './parm_rst/closed6_dry_rot4.parm7')
rf5=pt.load('./parm_rst/closed6_dry_rot5.rst7', './parm_rst/closed6_dry_rot5.parm7')

rfopen=pt.load('./parm_rst/open10_dry.rst7', './parm_rst/open10_dry.parm7')

## Free energy surface plot

List the trajectories

In [3]:
indir = './traj_dry_noeq_without_ipa'
topfile = './parm_rst/open10_dry.parm7'
from glob import glob
traj_list = sorted(glob(indir+'/*.nc'))
traj_list

['./traj_dry_noeq_without_ipa/closed_dry_1.nc',
 './traj_dry_noeq_without_ipa/closed_dry_10.nc',
 './traj_dry_noeq_without_ipa/closed_dry_2.nc',
 './traj_dry_noeq_without_ipa/closed_dry_3.nc',
 './traj_dry_noeq_without_ipa/closed_dry_4.nc',
 './traj_dry_noeq_without_ipa/closed_dry_5.nc',
 './traj_dry_noeq_without_ipa/closed_dry_6.nc',
 './traj_dry_noeq_without_ipa/closed_dry_7.nc',
 './traj_dry_noeq_without_ipa/closed_dry_8.nc',
 './traj_dry_noeq_without_ipa/closed_dry_9.nc',
 './traj_dry_noeq_without_ipa/open_dry_1.nc',
 './traj_dry_noeq_without_ipa/open_dry_10.nc',
 './traj_dry_noeq_without_ipa/open_dry_2.nc',
 './traj_dry_noeq_without_ipa/open_dry_3.nc',
 './traj_dry_noeq_without_ipa/open_dry_4.nc',
 './traj_dry_noeq_without_ipa/open_dry_5.nc',
 './traj_dry_noeq_without_ipa/open_dry_6.nc',
 './traj_dry_noeq_without_ipa/open_dry_7.nc',
 './traj_dry_noeq_without_ipa/open_dry_8.nc',
 './traj_dry_noeq_without_ipa/open_dry_9.nc']

Calculate the RMSD of each trajectory frames with one of teh reference structure. 
6 reference structures are used for the closed form in order to account to chain symmetry. Only the minimum RMSD value for each frame with respect to the closed crystal structures is kept

In [4]:
results_open=[]
results_closed=[]
for t in traj_list:
    traj=pt.load(t, topfile)
    rmsd0=pt.rmsd(traj,ref=rf0, mask="@CA,N,C,O")
    rmsd1=pt.rmsd(traj,ref=rf1, mask="@CA,N,C,O")
    rmsd2=pt.rmsd(traj,ref=rf2, mask="@CA,N,C,O")
    rmsd3=pt.rmsd(traj,ref=rf3, mask="@CA,N,C,O")
    rmsd4=pt.rmsd(traj,ref=rf4, mask="@CA,N,C,O")
    rmsd5=pt.rmsd(traj,ref=rf5, mask="@CA,N,C,O")
    
    rmsd_closed=np.nanmin(np.array([rmsd0,rmsd1,rmsd2,rmsd3,rmsd4,rmsd5]), axis=0)
    results_closed.append(rmsd_closed)
    
    rmsd_open=pt.rmsd(traj,ref=rfopen, mask="@CA,N,C,O")
    results_open.append(rmsd_open)

Create a Y matrix similar to that optained with PyEMMA after TICA analysis

In [42]:
Y=[]
for i in range(0,len(results_open)):  
    Z=[]
    for j in range(0,len(results_open[i])):
        a=np.array([results_closed[i][j],results_open[i][j]])
        Z.append(a)
    Z=np.array(Z)
    Y.append(Z)
print(len(Y))
print(len(Y[0]))
print(len(Y[0][0]))

20
48680
2


Plot interactive FES 

In [43]:
mpx_wdg_box = molpx.visualize.FES(traj_list, topfile, Y, nbins=200, n_sample= 500,
                                  proj_labels=[r'RMSD to closed crystal structure ($\AA$)',
                                               r'RMSD to open crystal structure ($\AA$)'])
mpx_wdg_box.linked_ngl_wdgs[0].camera = 'orthographic'
mpx_wdg_box

21-04-20 15:30:32 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[63] WARNING  Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
21-04-20 15:30:32 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[63] INFO     Presumably finished estimation. Message: Used data for centers: 9.93%


/home/el14718/Software/anaconda3_2019/lib/python3.7/site-packages/pyemma/coordinates/clustering/regspace.py:153: UserWarning: Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
  warnings.warn(msg)


MolPXHBox(children=(NGLWidget(max_frame=512), Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view'…

In [44]:
mpx_wdg_box.linked_ngl_wdgs[0].remove_line()
mpx_wdg_box.linked_ngl_wdgs[0].remove_licorice()
mpx_wdg_box.linked_ngl_wdgs[0].remove_surface()

In [45]:
mpx_wdg_box.linked_ngl_wdgs[0].add_line(selection="(not hydrogen) and (not backbone)")

In [ ]:
mpx_wdg_box.linked_ngl_wdgs[0].add_surface(selection="all", surfacetype="ms",probeRadius=0.8 )

K-mean clustering

In [9]:
n_clusters = 6
clustering = pyemma.coordinates.cluster_kmeans(Y,k=n_clusters, max_iter=1000, fixed_seed=True,
                                               n_jobs=1, init_strategy='uniform')

In [10]:
def plot_labels(ax=None):
    #if ax is None:
        #ax = gca()
    for i in range(0,len(clustering.clustercenters)):
        plt.text(clustering.clustercenters[i][0]+0.04, clustering.clustercenters[i][1]+0.04, 
                 i, fontsize=10, color='white')


In [11]:
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1])
cc_x = clustering.clustercenters[:, 0]
cc_y = clustering.clustercenters[:, 1]
plt.plot(cc_x, cc_y, linewidth=0, marker='o', markersize=5, color='white')
plot_labels()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Extract representative structures

In [12]:
data_sample, geoms = molpx.generate.sample(traj_list, topfile, clustering, 
                                                   n_geom_samples=100, n_points=100)

In [13]:
for i in range(0,len(clustering.clustercenters)):
    geoms[i].save('./rep_struct_1/{}.pdb'.format(i))

## Free energy surface for IPA containing simulations

In [14]:
indir = './traj_dry_noeq_with_ipa'
topfile = './parm_rst/open10_dry.parm7'
from glob import glob
traj_list2 = sorted(glob(indir+'/*.nc'))
traj_list2

['./traj_dry_noeq_with_ipa/closed_ipa_dry_1.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_10.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_2.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_3.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_4.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_5.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_6.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_7.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_8.nc',
 './traj_dry_noeq_with_ipa/closed_ipa_dry_9.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_1.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_10.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_2.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_3.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_4.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_5.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_6.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_7.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_8.nc',
 './traj_dry_noeq_with_ipa/open_ipa_dry_9.nc']

In [15]:
results_open_ipa=[]
results_closed_ipa=[]
for t in traj_list2:
    traj=pt.load(t, topfile)
    rmsd0=pt.rmsd(traj,ref=rf0, mask="@CA,N,C,O")
    rmsd1=pt.rmsd(traj,ref=rf1, mask="@CA,N,C,O")
    rmsd2=pt.rmsd(traj,ref=rf2, mask="@CA,N,C,O")
    rmsd3=pt.rmsd(traj,ref=rf3, mask="@CA,N,C,O")
    rmsd4=pt.rmsd(traj,ref=rf4, mask="@CA,N,C,O")
    rmsd5=pt.rmsd(traj,ref=rf5, mask="@CA,N,C,O")
    
    rmsd_closed_ipa=np.nanmin(np.array([rmsd0,rmsd1,rmsd2,rmsd3,rmsd4,rmsd5]), axis=0)
    results_closed_ipa.append(rmsd_closed_ipa)
    
    rmsd_open_ipa=pt.rmsd(traj,ref=rfopen, mask="@CA,N,C,O")
    results_open_ipa.append(rmsd_open_ipa)

In [46]:
Y_ipa=[]
for i in range(0,len(results_open_ipa)):  
    Z_ipa=[]
    for j in range(0,len(results_open_ipa[i])):
        a_ipa=np.array([results_closed_ipa[i][j],results_open_ipa[i][j]])
        Z_ipa.append(a_ipa)
    Z_ipa=np.array(Z_ipa)
    Y_ipa.append(Z_ipa)

In [47]:
mpx_wdg_box_ipa = molpx.visualize.FES(traj_list2, topfile, Y_ipa, nbins=200, n_sample= 500,
                                  proj_labels=[r'RMSD to closed crystal structure ($\AA$)',
                                               r'RMSD to open crystal structure ($\AA$)'])
mpx_wdg_box_ipa.linked_ngl_wdgs[0].camera = 'orthographic'
mpx_wdg_box_ipa

21-04-20 15:44:43 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[74] WARNING  Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
21-04-20 15:44:43 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[74] INFO     Presumably finished estimation. Message: Used data for centers: 5.00%


/home/el14718/Software/anaconda3_2019/lib/python3.7/site-packages/pyemma/coordinates/clustering/regspace.py:153: UserWarning: Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
  warnings.warn(msg)


MolPXHBox(children=(NGLWidget(max_frame=461), Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view'…

In [48]:
mpx_wdg_box_ipa.linked_ngl_wdgs[0].remove_line()
mpx_wdg_box_ipa.linked_ngl_wdgs[0].remove_licorice()
mpx_wdg_box_ipa.linked_ngl_wdgs[0].remove_surface()

In [44]:
mpx_wdg_box_ipa.linked_ngl_wdgs[0].add_surface(selection="all", surfacetype="ms",probeRadius=0.8 )

In [50]:
mpx_wdg_box.linked_ngl_wdgs[0].add_line(selection="(not hydrogen) and (not backbone)")

In [20]:
n_clusters = 4
clustering_ipa = pyemma.coordinates.cluster_kmeans(Y_ipa,k=n_clusters, max_iter=1000, fixed_seed=True,
                                               n_jobs=1, init_strategy='uniform')

In [21]:
def plot_labels(ax=None):
    #if ax is None:
        #ax = gca()
    for i in range(0,len(clustering_ipa.clustercenters)):
        plt.text(clustering_ipa.clustercenters[i][0]+0.04, clustering_ipa.clustercenters[i][1]+0.04, 
                 i, fontsize=10, color='white')


In [22]:
pyemma.plots.plot_free_energy(np.vstack(Y_ipa)[:, 0], np.vstack(Y_ipa)[:, 1])
cc_ipa_x = clustering_ipa.clustercenters[:, 0]
cc_ipa_y = clustering_ipa.clustercenters[:, 1]
plt.plot(cc_ipa_x, cc_ipa_y, linewidth=0, marker='o', markersize=5, color='white')
plot_labels()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
clustering_ipa

KmeansClustering(clustercenters=array([[3.58863, 3.15464],
       [2.98035, 1.93679],
       [2.12217, 2.95009],
       [2.63718, 2.66528]], dtype=float32),
         fixed_seed=42, init_strategy='uniform', keep_data=False,
         max_iter=1000, metric='euclidean', n_clusters=4, n_jobs=1,
         oom_strategy='memmap', skip=0, stride=1, tolerance=1e-05)

In [24]:
data_sample_ipa, geoms_ipa = molpx.generate.sample(traj_list2, topfile, clustering_ipa, 
                                                   n_geom_samples=500, n_points=1000)

In [25]:
for i in range(0,len(clustering_ipa.clustercenters)):
    geoms_ipa[i].save('./rep_struct_ipa_1/{}.pdb'.format(i))

In [61]:
clkmeans = pyemma.coordinates.cluster_kmeans([iY[:,:2] for iY in Y_ipa], 4, max_iter=1000, n_jobs=1, init_strategy='uniform')

In [62]:
clkmeans

KmeansClustering(clustercenters=array([[2.63718, 2.66528],
       [2.98035, 1.93679],
       [3.58863, 3.15464],
       [2.12217, 2.95008]], dtype=float32),
         fixed_seed=1420792865, init_strategy='uniform', keep_data=False,
         max_iter=1000, metric='euclidean', n_clusters=4, n_jobs=1,
         oom_strategy='memmap', skip=0, stride=1, tolerance=1e-05)

In [63]:
def plot_labels(ax=None):
    #if ax is None:
        #ax = gca()
    for i in range(0,len(clkmeans.clustercenters)):
        plt.text(clkmeans.clustercenters[i][0]+0.04, clkmeans.clustercenters[i][1]+0.04, 
                 i, fontsize=10, color='white')

In [64]:
pyemma.plots.plot_free_energy(np.vstack(Y_ipa)[:, 0], np.vstack(Y_ipa)[:, 1])
cc_ipa_x = clkmeans.clustercenters[:, 0]
cc_ipa_y = clkmeans.clustercenters[:, 1]
plt.plot(cc_ipa_x, cc_ipa_y, linewidth=0, marker='o', markersize=5, color='white')
plot_labels()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
data_sample, geoms = molpx.generate.sample(traj_list2, topfile, clkmeans,n_geom_samples=1000, n_points=10000)

In [69]:
for i in range(0,len(clkmeans.clustercenters)):
    geoms[i].save('./rep_struct_ipa_1/{}.pdb'.format(i))

In [ ]:
rmsd_open=np.concatenate(results_open, axis=0)
rmsd_closed=np.concatenate(results_closed, axis=0)

In [ ]:
plt.plot(rmsd_closed, rmsd_open, marker=".", ms=0.5, ls='None') 

In [ ]:
pyemma.plots.plot_free_energy(rmsd_closed, rmsd_open, nbins=200)

In [ ]:
n_clusters = 8
clustering = pyemma.coordinates.cluster_kmeans(Y,k=n_clusters, max_iter=1000, n_jobs=1)

In [ ]:
clustering.clustercenters

In [ ]:
def plot_labels(ax=None):
    for i in range(0,len(clustering.clustercenters)):
        plt.text(clustering.clustercenters[i][0]+0.06, clustering.clustercenters[i][1]+0.07, 
                 i, fontsize=10, color='black')

In [ ]:
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1], nbins=200)
cc_x = clustering.clustercenters[:, 0]
cc_y = clustering.clustercenters[:, 1]
plt.plot(cc_x, cc_y, linewidth=0, marker='o', markersize=5, color='white')
plot_labels()

In [ ]:
import molpx
import mdtraj as md

In [ ]:
indir = './Trajectories_all'
topfile =  indir+'/open10_dry.parm7'
from glob import glob
traj_list = sorted(glob(indir+'/*.nc'))
traj_list

In [ ]:
mpx_wdg_box = molpx.visualize.FES(traj_list,
                                 #MD_trajfiles,
                                 topfile,
                                 Y,
                                 #Y,
                                 nbins=50,
                                 #proj_idxs=[1,2],
                                 #proj_labels='RMSD',
                                 #n_overlays=5,
                                )
mpx_wdg_box